### Install packages

Run `pip install -r requirements.txt` to install the required packages.

### prepare training data/images
loop over images in `/sample/original` to crop them to just the gate for training

In [ ]:
from PIL import Image
import glob
import shutil

# get all JPG images from the ./sample/original folder
original_jpg_files = glob.glob('./sample/original/*.jpg')

# clear cropped folder
shutil.rmtree('./sample/cropped', ignore_errors=True)

# make cropped folder
shutil.os.mkdir('./sample/cropped')

for index, file in enumerate(original_jpg_files):
    im = Image.open(file)

    # dimensions of just the gate in the image
    left = 2142
    top = 333
    right = left + 585
    bottom = top + 468

    im1 = im.crop((2142, 333, right, bottom))

    # save the cropped image in `cropped` folder, create folder if necessary
    im1.save(file.replace('original', 'cropped'))

    print("processed {} of {}".format(index + 1, len(original_jpg_files)))

send cropped to Roboflow for labelling

In [ ]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Initialize the Roboflow object with your API key
rf = Roboflow(api_key=os.environ.get("ROBOFLOW_API_KEY"))

# Retrieve your current workspace and project name
print(rf.workspace())

# Specify the project for upload
project = rf.workspace(os.environ.get("ROBOFLOW_WORKSPACE")).project(os.environ.get("ROBOFLOW_PROJECT"))

cropped_jpg_files = glob.glob('./sample/cropped/*.jpg')

for index, file in enumerate(cropped_jpg_files):
    # Upload the image to your project
    project.upload(file)

    print("uploaded {} of {}".format(index + 1, len(cropped_jpg_files)))

### after labelling
download training set from Roboflow

In [ ]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv
import shutil

load_dotenv()  # take environment variables from .env.

rf = Roboflow(api_key=os.environ.get("ROBOFLOW_API_KEY"))
project = rf.workspace(os.environ.get("ROBOFLOW_WORKSPACE")).project(os.environ.get("ROBOFLOW_PROJECT"))
versions = project.versions()
latestVersion = versions[0]

print("latest version: {}".format(latestVersion))

dataset = project.version(latestVersion.version).download("folder")

# YOLO expects validation data to be in a folder named "val"
# whereas Roboflow names it "valid"
# delete any existing folder
if os.path.exists(dataset.location + "/valid"):
    shutil.move(dataset.location + "/valid", dataset.location + "/val")


train yolov with the custom dataset

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11x-cls.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(
    data=dataset.location, epochs=100,
    # adjust batch size based on GPU memory
    batch=300,
    # don't early exit, finish training
    patience=0,
    # disable random augmentations
    fliplr=0, scale=0, erasing=0,
)

validate data using trained weight

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO(str(results.save_dir)+'/weights/best.pt')

source = f'{dataset.location}/test/**/*.jpg'

# Run inference on the source
validateResults = model(source, stream=True,)  # generator of Results objects

for result in validateResults:
    print(result.probs.top1)

export ONNX nodel

In [ ]:
from ultralytics import YOLO

model = YOLO(str(results.save_dir)+'/weights/best.pt')
model.export(format='onnx')

upload trained weights to roboflow

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# model not supported by Roboflow
latestVersion.deploy("yolov11", results.save_dir)